# GlandsFinder #

In [1]:
from PIL import Image
from PIL import ImageFile
from PIL import Image
import PIL
PIL.Image.MAX_IMAGE_PIXELS = None
from ultralytics import YOLO

In [8]:
import os

# Sprawdzanie, czy plik istnieje w zadanej ścieżce
path = '../preprocessedData/LearnSet/data.yaml'
print("Plik istnieje?", os.path.exists(path))


Plik istnieje? True


In [13]:
model = YOLO("yolo11n.pt")
model.train(
    data='../preprocessedData/LearnSet145/data.yaml',
    epochs=50,
    imgsz=640,
    project="runs/train",
    name="Glands_Finder",
    exist_ok=True
)

New https://pypi.org/project/ultralytics/8.3.107 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.103  Python-3.12.4 torch-2.6.0+cpu CPU (12th Gen Intel Core(TM) i7-12700H)
engine\trainer: task=detect, mode=train, model=yolo11n.pt, data=../preprocessedData/LearnSet145/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=runs/train, name=Glands_Finder, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False,

train: Scanning C:\Users\stszy\jupyterNoteBookNotes\cellDetector\preprocessedData\LearnSet145\labels\train.cache... 19 

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning C:\Users\stszy\jupyterNoteBookNotes\cellDetector\preprocessedData\LearnSet145\labels\train.cache... 19 im

Plotting labels to runs\train\Glands_Finder\labels.jpg... 


...

                   all         19        400      0.871      0.927      0.962       0.87



50 epochs completed in 0.084 hours.
Optimizer stripped from runs\train\Glands_Finder\weights\last.pt, 5.5MB
Optimizer stripped from runs\train\Glands_Finder\weights\best.pt, 5.5MB

Validating runs\train\Glands_Finder\weights\best.pt...
Ultralytics 8.3.103  Python-3.12.4 torch-2.6.0+cpu CPU (12th Gen Intel Core(TM) i7-12700H)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<0


                   all         19        400      0.871      0.927      0.962       0.87
Speed: 0.6ms preprocess, 38.2ms inference, 0.0ms loss, 15.0ms postprocess per image
Results saved to runs\train\Glands_Finder


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000015F6A1037D0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

In [17]:
img = Image.open("../preprocessedData/patient0.tif")
results = model(img, conf=0.05)


0: 640x640 79 glands, 80.6ms
Speed: 5.7ms preprocess, 80.6ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)


In [18]:
results[0].show()

In [16]:
im_array = results[0].plot()  
im = Image.fromarray(im_array)
im.save("output.tif", format="TIFF")

# Augumentation #

In [21]:
import os
import cv2
import albumentations as A

In [ ]:
def load_labels(label_path):
    bboxes = []
    class_labels = []
    with open(label_path, 'r') as f:
        for line in f.readlines():
            parts = line.strip().split()
            class_id = int(parts[0])
            bbox = list(map(float, parts[1:]))
            bboxes.append(bbox)
            class_labels.append(class_id)
    return bboxes, class_labels

In [ ]:
def save_augmented_data(image, bboxes, class_labels, out_img_path, out_label_path):
    cv2.imwrite(out_img_path, image)
    with open(out_label_path, 'w') as f:
        for cls, bbox in zip(class_labels, bboxes):
            f.write(f"{cls} {' '.join(map(str, bbox))}\n")

In [25]:
transformations = [
    A.Compose([A.Rotate(limit=(0, 0), p=1)], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels'])),
    A.Compose([A.Rotate(limit=(90, 90), p=1)], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels'])),
    A.Compose([A.Rotate(limit=(180, 180), p=1)], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels'])),
    A.Compose([A.Rotate(limit=(270, 270), p=1)], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels'])),
]

flip = A.Compose([
    A.HorizontalFlip(p=1)
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

images_dir = '../preprocessedData/LearnSet/data/images/train'
labels_dir =  '../preprocessedData/LearnSet/data/labels/train'
output_images_dir = '../preprocessedData/LearnSet/dataAugumented/images/train'
output_labels_dir = '../preprocessedData/LearnSet/dataAugumented/labels/train'

os.makedirs(images_dir, exist_ok=True)
os.makedirs(labels_dir, exist_ok=True)
os.makedirs(output_images_dir, exist_ok=True)
os.makedirs(output_labels_dir, exist_ok=True)

for filename in os.listdir(images_dir):
    if not filename.lower().endswith(('.jpg', '.png', '.jpeg', '.tif', '.tiff')):
        continue

    img_path = os.path.join(images_dir, filename)
    label_path = os.path.join(labels_dir, filename.rsplit('.', 1)[0] + '.txt')

    image = cv2.imread(img_path)
    if image is None:
        print(f"Cannot read image: {img_path}")
        continue

    bboxes, class_labels = load_labels(label_path)
    if not bboxes:
        continue

    for i, rot in enumerate(transformations):
        result = rot(image=image, bboxes=bboxes, class_labels=class_labels)
        out_name = f"{filename.rsplit('.', 1)[0]}_rot{i}"
        save_augmented_data(
            result['image'],
            result['bboxes'],
            result['class_labels'],
            os.path.join(output_images_dir, out_name + '.tif'),
            os.path.join(output_labels_dir, out_name + '.txt')
        )

        flipped = flip(image=result['image'], bboxes=result['bboxes'], class_labels=result['class_labels'])
        out_name_flip = f"{filename.rsplit('.', 1)[0]}_rot{i}_flip"
        save_augmented_data(
            flipped['image'],
            flipped['bboxes'],
            flipped['class_labels'],
            os.path.join(output_images_dir, out_name_flip + '.tif'),
            os.path.join(output_labels_dir, out_name_flip + '.txt')
        )


# Model trainig with augumented Data #

In [28]:
model = YOLO("yolo11n.pt")
model.train(
    data='../preprocessedData/LearnSet/dataAugumented.yaml',
    epochs=50,
    imgsz=640,
    project="runs/train",
    name="Glands_Finder",
    exist_ok=True
)

New https://pypi.org/project/ultralytics/8.3.107 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.103  Python-3.12.4 torch-2.6.0+cpu CPU (12th Gen Intel Core(TM) i7-12700H)
engine\trainer: task=detect, mode=train, model=yolo11n.pt, data=../preprocessedData/LearnSet/dataAugumented.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=runs/train, name=Glands_Finder, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks

train: Scanning C:\Users\stszy\jupyterNoteBookNotes\cellDetector\preprocessedData\LearnSet\dataAugumented\labels\train.


train: New cache created: C:\Users\stszy\jupyterNoteBookNotes\cellDetector\preprocessedData\LearnSet\dataAugumented\labels\train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Scanning C:\Users\stszy\jupyterNoteBookNotes\cellDetector\preprocessedData\LearnSet\dataAugumented\labels\train.ca

Plotting labels to runs\train\Glands_Finder\labels.jpg... 


...

                   all        152       3200      0.996      0.999      0.995      0.954



50 epochs completed in 0.758 hours.
Optimizer stripped from runs\train\Glands_Finder\weights\last.pt, 5.5MB
Optimizer stripped from runs\train\Glands_Finder\weights\best.pt, 5.5MB

Validating runs\train\Glands_Finder\weights\best.pt...
Ultralytics 8.3.103  Python-3.12.4 torch-2.6.0+cpu CPU (12th Gen Intel Core(TM) i7-12700H)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:18<0


                   all        152       3200      0.996      0.998      0.995      0.954
Speed: 0.8ms preprocess, 53.6ms inference, 0.0ms loss, 7.8ms postprocess per image
Results saved to runs\train\Glands_Finder


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000015F5CC17EC0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

In [29]:
img = Image.open("../preprocessedData/patient0.tif")
results = model(img, conf=0.05)


0: 640x640 82 glands, 68.0ms
Speed: 3.5ms preprocess, 68.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


In [30]:
results[0].show()